<a href="https://colab.research.google.com/github/kabirodavies/ML_classification/blob/main/Climate_Change_Belief_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# How do machines understand language?

© Explore Data Science Academy

In [34]:
# pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [3]:
import nltk

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import re
from textblob import TextBlob
from wordcloud import WordCloud
# set plot style
sns.set()

In [4]:
from google.colab import files
uploaded = files.upload()

Saving sample_submission.csv to sample_submission.csv
Saving test.csv to test.csv
Saving train.csv to train.csv


## Let's see out data

In [17]:
df = pd.read_csv('train.csv')
df.head()


,sentiment,message,tweetid
0,1,PolySciMajor EPA chief doesn't think carbon di...,625221
1,1,It's not like we lack evidence of anthropogeni...,126103
2,2,RT @RawStory: Researchers say we have three ye...,698562
3,1,#TodayinMaker# WIRED : 2016 was a pivotal year...,573736
4,1,"RT @SoyNovioDeTodas: It's 2016, and a racist, ...",466954


In [21]:
sub = pd.read_csv('sample_submission.csv')
sub["sentiment"].value_counts()
# sub.head(100)

1    10546
Name: sentiment, dtype: int64

In [28]:
new = df[df['sentiment']==-1]
new

,sentiment,message,tweetid
28,-1,Sally Kohn’s latest evidence of climate change...,355491
46,-1,@realDonaldTrump Carbon Tax is a Globalist ide...,61141
48,-1,RT @SteveSGoddard: We had winds close to 100 M...,719523
56,-1,@MissLizzyNJ lmao 😂 snowflakes ❄️ complaining ...,911385
57,-1,RT @Dawn2334Dawn: This is ONE of Arnold Schwar...,768263
...,...,...,...
15704,-1,RT @dbongino: We're at the point w/the delusio...,890582
15705,-1,RT @joshdcaplan: Al Gore admits Paris Accord w...,380307
15711,-1,RT @JoshNoneYaBiz: Funny the same people who b...,390339
15806,-1,RT @HerberMp: @sness5561_ness Al Gore is a idi...,213863


In [32]:
asa = sub[df['sentiment']==2]
asa

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


,tweetid,sentiment
2,224985,1
12,143459,1
14,20476,1
26,588985,1
32,205796,1
...,...,...
10533,959088,1
10535,940588,1
10537,961513,1
10539,27319,1


In [7]:
df.columns

Index(['sentiment', 'message', 'tweetid'], dtype='object')

##NLTK

In [ ]:
nltk.download(['punkt','stopwords'])

In [ ]:
from nltk.corpus import stopwords

In [ ]:
stopwords_list = stopwords.words('english')
print(stopwords_list)

## Data Cleaning
Step 1: Convert to lower text

In [ ]:
df['message'] = df['message'].str.lower()
df.head()

Step 2: Remove the punctuation using the `string` import.

In [ ]:
# import string
# # print(string.punctuation)

In [ ]:
# def remove_punctuation(post):
#     return ''.join([l for l in post if l not in string.punctuation])

In [ ]:
# df['message'] = df['message'].apply(remove_punctuation)
df.head()

In [ ]:
def clean_text(text):
  # text = text.lower() #Convert all text to lowercase
  text = re.sub(r'@[A-Za-z0-9]+', '', text) # Remove @mentions
  text = re.sub(r'#', '', text) # Remove '#' symbol
  text = re.sub(r':', '', text) # Remove ':' symbol
  text = re.sub(r'rt[\s]+', '', text) #Remove RT
  text = re.sub(r'https?:\/\/\S+', '', text) # Remove hyperlink
  text = re.sub(r'\w*\d\w*', '', text) # Remove numbers


  return text

#Cleaning the text

In [ ]:
df["message"] = df["message"].apply(clean_text)

In [ ]:
# pattern_url = r'http[s]?://(?:[A-Za-z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9A-Fa-f][0-9A-Fa-f]))+'
# subs_url = r'url-web'
# df['message'] = df['message'].replace(to_replace = pattern_url, value = subs_url, regex = True)

#Show cleaned text

In [ ]:
df_clean = df.copy()
df_clean.head()

In [ ]:
# Function to get subjectivity
def getSubjectivity(text):
  return TextBlob(text).sentiment.subjectivity

# Function to get polarity
def getPolarity(text):
  return TextBlob(text).sentiment.polarity
  
#Create two columns
df_clean['subjectivity'] = df_clean['message'].apply(getSubjectivity)
df_clean['polarity'] = df_clean['message'].apply(getPolarity)
df_clean

In [ ]:
#Visualise Word Cloud
allwords = ' '.join( [twts for twts in df_clean['message']] )
wordCloud = WordCloud(width = 500, height = 300, random_state = 21, max_font_size = 119).generate(allwords)

plt.imshow(wordCloud, interpolation='bilinear')
plt.axis('off')
plt.show()

In [ ]:
# df_clean['message']

## Tokenisation

In [ ]:
from nltk.tokenize import word_tokenize, TreebankWordTokenizer
from nltk.tokenize import TweetTokenizer

In [ ]:
tokeniser = TweetTokenizer()
df['message'] = df['message'].apply(tokeniser.tokenize)
df

In [ ]:
words = df['message']

In [ ]:
word_len = [len(w) for w in words]
plt.hist(word_len)

## Stemming

In [ ]:
from nltk import SnowballStemmer, PorterStemmer, LancasterStemmer

In [ ]:
def mbti_stemmer(words, stemmer):
    return [stemmer.stem(word) for word in words]

In [ ]:
stemmer = SnowballStemmer('english')
df['message'] = df['message'].apply(mbti_stemmer, args=(stemmer, ))

### Lemmatization

In [ ]:
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')

lemmatizer = WordNetLemmatizer()

In [ ]:
def mbti_lemma(words, lemmatizer):
    return [lemmatizer.lemmatize(word) for word in words]  

In [ ]:
df['message'] = df['message'].apply(mbti_lemma, args=(lemmatizer, ))

### Stop Words

In [ ]:
from nltk.corpus import stopwords
print(stopwords.words('english'))

In [ ]:
sorted(stopwords.words('english'))[0:10]

In [ ]:
def remove_stop_words(tokens):    
    return [t for t in tokens if t not in stopwords.words('english')]

In [ ]:
df['message'] = df['message'].apply(remove_stop_words)

In [ ]:
df

In [ ]:
df["sentiment"].value_counts()

## Text feature extraction

### Bag of words

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
vect = CountVectorizer()
vect.fit(df_clean['message'])

## Frequency Distribution


In [ ]:
from nltk.probability import FreqDist
fdist = FreqDist(df_clean['message'])
print(fdist)

In [ ]:
fdist.most_common(2)

#Performing Sentiment Analysis using Text Classification

In [ ]:
df_clean.head()

In [ ]:
df_clean.info()

In [ ]:
df_clean.sentiment.value_counts()

In [ ]:
sentiment_count = df_clean.groupby('sentiment').count()
plt.bar(sentiment_count.index.values, sentiment_count['message'])
plt.xlabel('Review Sentiments')
plt.ylabel('Number of Review')
plt.show()

##Feature Generation using Bag of Words

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from nltk.tokenize import RegexpTokenizer
#tokenizer to remove unwanted elements from out data like symbols and numbers
token = RegexpTokenizer(r'[a-zA-Z0-9]+')
cv = CountVectorizer(lowercase=True,stop_words='english',ngram_range = (1,1),tokenizer = token.tokenize)
text_counts= cv.fit_transform(df_clean['message'])

##Split train and test set


In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    text_counts, df_clean['sentiment'], test_size=0.3, random_state=1)

In [ ]:
from sklearn.naive_bayes import MultinomialNB
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
# Model Generation Using Multinomial Naive Bayes
clf = MultinomialNB().fit(X_train, y_train)
predicted= clf.predict(X_test)
print("MultinomialNB Accuracy:",metrics.accuracy_score(y_test, predicted))

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tf=TfidfVectorizer()
text_tf= tf.fit_transform(df_clean['message'])

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    text_tf, df_clean['message'], test_size=0.3, random_state=123)

In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
# Model Generation Using Multinomial Naive Bayes
clf = MultinomialNB().fit(X_train, y_train)
predicted= clf.predict(X_test)
print("MultinomialNB Accuracy:",metrics.accuracy_score(y_test, predicted))